In [3]:
search_for = 126
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0021195411682128906
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 956096141
type: [1, 1, 1, 1, 126] 126
cases of this type: 252047376
100000 21.712839074936664
200000 34.33716700708731
300000 35.48616118161923
400000 45.91245270552264
500000 50.364777368042375
600000 52.93444003218066
700000 49.30866325807447
800000 49.68888914723112
900000 54.43206728159194
1000000 37.45279828901357
1100000 33.62982004096
1200000 57.619998844536354
1300000 52.013317319177084
1400000 58.4765389884453
1500000 54.55569071291244
1600000 62.92699709053038
1700000 55.957351450391606
1800000 61.83673616078767
1900000 56.1238219433531
2000000 49.399088945367616
2100000 31.094326530267303
2200000 54.01533678522711
2300000 52.38632512004314
2400000 44.076222133898746
2500000 53.542424452083594
2600000 52.824372640469555
2700000 48.61179145263703
2800000 57.185825090281774
2900000 50.062104626492086
3000000 30.3043404

30000000 75.14560742849577
30100000 77.51766874439016
30200000 74.35644993282146
30300000 31.463266023428858
30400000 77.53942982278818
30500000 78.04703421711304
30600000 74.99982319603247
30700000 75.87721838419444
30800000 71.92788343721092
30900000 70.43980352943937
31000000 56.2336141189149
31100000 40.97103751910335
31200000 70.1649289687574
31300000 71.50159545649124
31400000 70.62081674335292
31500000 74.24163061235782
31600000 75.91762892899284
31700000 77.00119615288197
31800000 42.77338731789395
31900000 72.5434299646303
32000000 76.82325793775745
32100000 74.127066700101
32200000 72.77639465184981
32300000 34.96195785226487
32400000 71.01115517612861
32500000 73.7600257117489
32600000 72.50589388676903
32700000 68.74844872179365
32800000 69.28601110154678
32900000 66.2266013414245
33000000 51.628472389532185
33100000 40.80302128303807
33200000 65.76602207718479
33300000 66.997437478727
33400000 67.8293345138086
33500000 71.8747801899973
33600000 73.28995037453612
33700000 7

60500000 47.10932824069255
60600000 50.599176722250206
60700000 59.816713066338515
60800000 61.75067863773459
60900000 63.09251602646934
61000000 54.057997977113544
61100000 34.16017938275659
61200000 62.58986652563967
61300000 62.525742823277085
61400000 61.9206834531211
61500000 63.13089843281165
61600000 32.60754000950174
61700000 67.37067093591412
61800000 67.86620748870916
61900000 69.80277517338362
62000000 75.9628969939711
62100000 79.21875894292643
62200000 74.65901956639928
62300000 73.72127955910946
62400000 70.03940951927471
62500000 61.127913215474905
62600000 41.776405562659406
62700000 65.89506002997653
62800000 70.30381317992692
62900000 67.24866035593004
63000000 59.517954006657725
63100000 35.68157290944661
63200000 65.8851758101689
63300000 66.33424099991927
63400000 64.49847799097849
63500000 61.03107318102583
63600000 39.80137948113426
63700000 68.20783485027091
63800000 72.51389179649937
63900000 69.5693234594133
64000000 75.9175214182878
64100000 74.62020040974915

90800000 27.751428586826687
90900000 61.0367726190387
91000000 59.631170444558656
91100000 31.380214981027475
91200000 58.466055698611186
91300000 52.97833100731123
91400000 43.33192655881248
91500000 61.05638665641003
91600000 63.292873340220794
91700000 62.22657056984081
91800000 70.11620341688096
91900000 68.84948099599883
92000000 68.4526068990979
92100000 67.3158093757149
92200000 64.67914395259925
92300000 64.0427906771764
92400000 64.96610724594774
92500000 62.318203895224414
92600000 62.31638108154516
92700000 61.83802587113695
92800000 26.91806391968225
92900000 58.57157889317505
93000000 53.089604918894544
93100000 30.641871437124983
93200000 57.83330725990787
93300000 44.25736260112301
93400000 46.94563033029251
93500000 57.801591685589536
93600000 63.253338697822365
93700000 59.67320769441774
93800000 64.36599610693077
93900000 65.09540281731456
94000000 65.29982028570038
94100000 67.38461848731657
94200000 68.54332080405241
94300000 68.20047212227499
94400000 65.0532388521

120200000 73.44569193629997
120300000 74.90336444143496
120400000 73.17833767661125
120500000 73.83120061320274
120600000 72.04263319915546
120700000 72.9039486573565
120800000 72.1339710088109
120900000 74.89120921937439
121000000 38.14057631062175
121100000 10.583439966080707
121200000 62.39379483934453
121300000 69.66352137216816
121400000 68.49514153452152
121500000 74.30502985160699
121600000 73.11232171950874
121700000 70.93830789388981
121800000 73.26254918513825
121900000 74.42422508284723
122000000 81.2373440040003
122100000 24.276689692101964
122200000 7.1985529117570035
122300000 6.652979467687781
122400000 6.612347763825933
122500000 6.918348705677401
122600000 6.71885300021464
122700000 6.166766692882157
122800000 6.949732679921318
122900000 6.470839416628784
123000000 6.491157930045362
123100000 7.4426631575127
123200000 6.809261509634881
123300000 6.1459594814298875
123400000 6.116459423379033
123500000 6.279584190945029
123600000 6.287677884513185
123700000 6.5484151019

...

884300000 2.4637807678482653
884400000 2.755963835763919
884500000 2.1655115435811148
884600000 2.9954537105055032
884700000 2.1023651332646063
884800000 3.327737979590516
884900000 1.9185712339266559
885000000 2.7882318301989
885100000 2.2443999406094477
885200000 2.83884687273742
885300000 2.6596832771253034
885400000 2.206388401094865
885500000 2.8900839344656326
885600000 1.8551133031005707
885700000 2.9923701725744993
885800000 1.4862281519430836
885900000 3.0490358671227056
886000000 1.540013137090782
886100000 2.7812283794190407
886200000 2.399600888203063
886300000 2.300129016448089
886400000 3.5457498731052897
886500000 2.222276610554857
886600000 1.532399705480415
886700000 0.47804176455948555
886800000 0.511390106619617
886900000 0.5011976489479422
887000000 0.4381543076948405
887100000 0.4828401955964433
887200000 0.47144739456365437
887300000 0.4923161566329658
887400000 0.4837622760153963
887500000 0.470052805737614
887600000 0.4876868529391388
887700000 2.346432280821642

909000000 0.3925931652632885
909100000 0.37513239687309796
909200000 0.31920917876792754
best so far: 0
type: [1, 3, 6, 7, 1] 126
cases of this type: 37044
best so far: 0
type: [1, 3, 7, 1, 6] 126
cases of this type: 190512
909300000 0.7174718073160589
909400000 0.3995664972102755
best so far: 0
type: [1, 3, 7, 2, 3] 126
cases of this type: 95256
909500000 0.2999216294916272
best so far: 0
type: [1, 3, 7, 3, 2] 126
cases of this type: 63504
909600000 0.26144106634061004
best so far: 0
type: [1, 3, 7, 6, 1] 126
cases of this type: 31752
best so far: 0
type: [1, 3, 14, 1, 3] 126
cases of this type: 47628
909700000 0.2948317533450637
best so far: 0
type: [1, 3, 14, 3, 1] 126
cases of this type: 15876
best so far: 0
type: [1, 3, 21, 1, 2] 126
cases of this type: 21168
best so far: 0
type: [1, 3, 21, 2, 1] 126
cases of this type: 10584
best so far: 0
type: [1, 3, 42, 1, 1] 126
cases of this type: 5292
best so far: 0
type: [1, 6, 1, 1, 21] 126
cases of this type: 1166886
909800000 0.75909246

920600000 1.1981358641612767
920700000 0.8686413369582587
920800000 1.2774479835307069
920900000 0.9102408406041841
921000000 1.0435958252192246
921100000 1.2310410011856185
921200000 0.8708093881698965
921300000 1.1548690580310803
921400000 0.8898362274061243
921500000 1.1238592706177672
921600000 1.2915247578575055
921700000 0.8720167936044627
921800000 1.3395274268590807
921900000 0.9006648384667979
922000000 1.1969546246629292
922100000 0.3654034788828373
922200000 0.24760575042072466
922300000 0.2426835992512683
922400000 0.24568248346884847
922500000 0.24278047422414223
922600000 0.24387395869961448
922700000 0.2413940407361448
922800000 0.2195960642401907
922900000 0.20096664751111604
923000000 0.23829572289469242
923100000 0.23802390667540893
923200000 0.23686822879335018
923300000 0.6525859767593404
923400000 1.0206106093027976
923500000 1.091355235077338
923600000 1.0671929063074668
923700000 0.7189866968520482
923800000 1.2544224505460275
923900000 0.8865851404894709
9240000

944400000 0.21265864914610982
944500000 0.07668276416492992
944600000 0.10699382720589638
944700000 0.19402604623993436
best so far: 0
type: [2, 3, 1, 3, 7] 126
cases of this type: 194481
944800000 0.17943531816333402
944900000 0.08767700193643371
best so far: 0
type: [2, 3, 1, 7, 3] 126
cases of this type: 83349
945000000 0.06760465760890642
best so far: 0
type: [2, 3, 1, 21, 1] 126
cases of this type: 27783
best so far: 0
type: [2, 3, 3, 1, 7] 126
cases of this type: 64827
945100000 0.055408671660207374
best so far: 0
type: [2, 3, 3, 7, 1] 126
cases of this type: 9261
best so far: 0
type: [2, 3, 7, 1, 3] 126
cases of this type: 11907
best so far: 0
type: [2, 3, 7, 3, 1] 126
cases of this type: 3969
best so far: 0
type: [2, 3, 21, 1, 1] 126
cases of this type: 1323
best so far: 0
type: [2, 7, 1, 1, 9] 126
cases of this type: 45927
best so far: 0
type: [2, 7, 1, 3, 3] 126
cases of this type: 15309
945200000 0.0985957920153161
best so far: 0
type: [2, 7, 1, 9, 1] 126
cases of this type:

best so far: 0
type: [3, 7, 6, 1, 1] 126
cases of this type: 252
best so far: 0
type: [3, 14, 1, 1, 3] 126
cases of this type: 1134
best so far: 0
type: [3, 14, 1, 3, 1] 126
cases of this type: 378
best so far: 0
type: [3, 14, 3, 1, 1] 126
cases of this type: 126
best so far: 0
type: [3, 21, 1, 1, 2] 126
cases of this type: 336
best so far: 0
type: [3, 21, 1, 2, 1] 126
cases of this type: 168
best so far: 0
type: [3, 21, 2, 1, 1] 126
cases of this type: 84
best so far: 0
type: [3, 42, 1, 1, 1] 126
cases of this type: 42
best so far: 0
type: [6, 1, 1, 1, 21] 126
cases of this type: 194481
955400000 0.005646354973000289
955500000 0.003778998007245858
best so far: 0
type: [6, 1, 1, 3, 7] 126
cases of this type: 64827
best so far: 0
type: [6, 1, 1, 7, 3] 126
cases of this type: 27783
955600000 0.0029903215619113714
best so far: 0
type: [6, 1, 1, 21, 1] 126
cases of this type: 9261
best so far: 0
type: [6, 1, 3, 1, 7] 126
cases of this type: 21609
best so far: 0
type: [6, 1, 3, 7, 1] 126
ca